In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.data import make_dataset
from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANLIZAR = 112103  # Este es el Torax

## 1. Obtencion de Métricas de Egresos Hospitalarios

En este apartado se quieren calculcar diversas métricas relevantes para el análisis de los
egresos hospitalarios en Chile. Las métricas a calcular son las siguientes:

1. Cantidad de Egresos Hospitalarios
2. Dias de Estada Totales
3. Cantidad de Intervenciones Quirúrgicas
4. Cantidad de Muertos

Todas estas métricas serán calculadas para cada Año, Establecimiento de Salud y Diagnóstico
distinto que exista en la base de datos. Por ejemplo:

"En el año 2019, el Hospital San José (código XXXXXX) tuvo XX egresos, con XX dias estada promedio,
con XX intervenciones quirúrgicas y XX muertos para el diagnóstico A24.4"s


In [3]:
df_nacional = pl.scan_csv(
    "../data/processed/egresos_procesados.csv",
    infer_schema_length=10000,
    null_values="Extranjero",
)

In [4]:
AGRUPACION = [
    "ANO_EGRESO",
    "ESTABLECIMIENTO_SALUD",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "DIAG1",
]

metricas = build_features.obtener_metricas_egresos(df_nacional, AGRUPACION).collect()

In [5]:
display(metricas.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos
i64,i64,str,str,u32,i64,i64,i64
2003,103219,"""Hospital del C…","""K719""",1,4,0,0
2015,125100,"""Hospital Regio…","""J358""",1,2,1,0
2008,111200,"""Hospital Clíni…","""T888""",1,2,0,0
2014,101100,"""Hospital Dr. J…","""M663""",3,6,3,0
2009,110100,"""Hospital San J…","""N340""",3,65,2,0
2019,111195,"""Hospital de Ur…","""K358""",407,583,396,0
2004,117201,"""Clínica Chillá…","""D175""",1,3,1,0
2004,109100,"""Complejo Hospi…","""N484""",1,3,0,0
2002,116100,"""Hospital San J…","""O926""",3,3,3,0


- Al ver una muestra del calculo realizado, se puede observar la cantidad de egresos, dias de
  estada, numero de intervenciones quirurgicas y cantidad de muertos para 10 duplas de
  hospitales - diagnosticos distintos.


Si bien se tienen las metricas calculadas, ahora se quiere responder a las siguientes preguntas:

- En qué diagnósticos se destaca cada hospital productivamente? Esto significa, en qué diagnósticos
  cada hospital tiene una mayor cantidad de egresos que otros hospitales?

- El hospital tiene un desempeño similar dentro de distintos grupos/estratos de hospitales?
  El hospital se desempeñara de la misma forma nacionalmente, dentro del estrato público, privado,
  etc...?

Para responder a la pregunta se realizará un ranking de todos los hospitales para cada
diagnóstico y cada año observado. Este ranking se realizará para distintos grupos/estratos de
hospitales (Nacional, Hospitales Públicos, Hospitales Privados, Hospitales GRD)


## 2. Análisis de Estratos

En este análisis se quiere observar el ranking de cada hospital/clínica del país para cada 
año y diagnóstico observado. Además, el ranking se realizará en los siguientes estratos:

1. Nacional
2. Públicos
3. Privados
4. Adscritos a GRD
5. INT

In [6]:
dict_estratos = build_features.obtener_diccionario_estratos(df_nacional, HOSPITAL_A_ANLIZAR)

In [35]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales = build_features.agregar_ranking_estratos(
    metricas, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

rankings_hospital_analizado = rankings_nacionales.filter(
    pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANLIZAR
)

In [61]:
display(rankings_nacionales.head(8))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2020,104103,"""Hospital Provi…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",6,32,2,0,1,15,0.4,1,14,0.428571,null,null,null,1,6,1.0,null,null,null
2020,125102,"""Hospital Dr. L…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",3,8,0,0,2,15,0.2,2,14,0.214286,null,null,null,null,null,null,null,null,null
2020,125103,"""Hospital Lord …","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",2,349,0,0,3,15,0.133333,3,14,0.142857,null,null,null,null,null,null,null,null,null
2020,118106,"""Hospital de Lo…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,116101,"""Hospital de Te…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,1,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,107107,"""Hospital de Pe…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,140,0,0,4,15,0.066667,4,14,0.071429,null,null,null,null,null,null,null,null,null
2020,102200,"""Clínica Iquiqu…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Personas en co…","""Z769""",1,3,1,0,4,15,0.066667,null,null,null,1,1,1.0,null,null,null,null,null,null
2020,112211,"""Clínica Indisa…","""Cap.21 FACTOR…","""Z69-Z76 PERSO…","""Z76 PERSONAS E…","""Persona en con…","""Z768""",58,118,14,1,1,299,0.19398,null,null,null,1,149,0.389262,null,null,null,null,null,null


Por lo tanto, se pueden observar todas las métricas de egresos hospitalarios nacionales para
cada estrato definido. Además, se observa que en casos de empate en cantidad de egresos,
el ranking es repetido.

## 3. Diagnósticos más relevantes del Instituto Nacional del Tórax

En este apartado se quieren responder las siguientes preguntas:

1. ¿Cuáles son los **diagnósticos más relevantes productivamente** en el Instituto Nacional del
Tórax?
2. ¿En qué diagnósticos el INT tiene una mayor cantidad de Egresos que cualquier otro establecimiento
de salud Nacional? En otras palabras, ¿En qué diagnósticos el **INT tiene un Ranking 1 Nacional**?

Estas preguntas se quieren responder tanto para diagnósticos ambulatorios (n_int_q = 0) y 
quirúrgicos (n_int_q > 0).

Además, se sabe que desde el 2011 el registro de Egresos Hospitalarios DEIS fue llevado por la
unidad de Grupo Relacionado al Diagnóstico (GRD). Debido a lo anterior, la calidad de la codificación
de diagnósticos es mucho mayor. Tomado de lo anterior, **se obtendrán los diagnósticos que tengan
una alta productividad desde el 2012 hasta 2020**.

In [73]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = rankings_hospital_analizado.filter(
    (pl.col("ranking_nacionales_n_egresos") == 1)
    & (pl.col("n_int_q") > 0)
    & (pl.col("ANO_EGRESO") > 2011)
).sort(by=["ANO_EGRESO", "n_egresos", "n_int_q"], descending=True)

veinte_mas_relevantes_por_anio = DIAGS_MAS_RELEVANTES_QUIRURGICOS.group_by(["ANO_EGRESO"]).head(20)

In [74]:
DIAGS_MAS_RELEVANTES_QUIRURGICOS = veinte_mas_relevantes_por_anio.select(
    pl.col("DIAG1").unique()
).sort(by="DIAG1")

In [78]:
print(list(DIAGS_MAS_RELEVANTES_QUIRURGICOS.to_series()))
print(f"Son {len(DIAGS_MAS_RELEVANTES_QUIRURGICOS)} diagnosticos quirurgicos")

['C33X', 'C340', 'C341', 'C342', 'C343', 'C381', 'C384', 'C450', 'C780', 'C782', 'D143', 'D381', 'E848', 'I080', 'I081', 'I340', 'I350', 'I351', 'I352', 'I420', 'I456', 'I472', 'I495', 'I710', 'I712', 'J398', 'J47X', 'J679', 'J841', 'J848', 'J849', 'J860', 'J869', 'J90X', 'J91X', 'J931', 'J955', 'J980', 'M348', 'Q211', 'Q231', 'Q676', 'T820', 'T821', 'Z450']
Son 45 diagnosticos quirurgicos


In [40]:
DIAGS_1_o_2_en_2019 = list((
    rankings_hospital_analizado.filter(
        (pl.col("DIAG1").is_in(DIAGS_MAS_RELEVANTES.to_series()))
        & (pl.col("ANO_EGRESO") == 2019)
        & (
            (pl.col("ranking_nacionales_n_egresos") == 1)
            | (pl.col("ranking_nacionales_n_egresos") == 2)
        )
    )
    .select(pl.col("DIAG1"))
).to_series())

DIAGS_1_o_2_en_2019 += ["T821", "I710", "I051", "D143"]

In [41]:
print(len(DIAGS_1_o_2_en_2019))

40


In [42]:
tabla_resumen_egresos_torax_relevantes = (
    rankings_hospital_analizado.filter(
        ((pl.col("ANO_EGRESO") >= 2016) & (pl.col("ANO_EGRESO") <= 2019))
        & (pl.col("DIAG1").is_in(DIAGS_1_o_2_en_2019))
    )
    .with_columns((pl.col("DIAG1") + " - " + pl.col("Descripción")).alias("Diagnóstico"))
    .to_pandas()
    .pivot(
        index=["Diagnóstico"],
        columns="ANO_EGRESO",
        values=[
            "ranking_nacionales_n_egresos",
            "n_egresos",
            "total_nacionales_n_egresos",
            "%_nacionales_n_egresos",
        ],
    )
).sort_values(
    by=[("ranking_nacionales_n_egresos", 2019), ("n_egresos", 2019)], ascending=[True, False]
)

In [43]:
resumen_egresos_torax_y_nacional_relevantes = (
    tabla_resumen_egresos_torax_relevantes[("n_egresos")].astype(int).astype(str)
    + " ("
    + tabla_resumen_egresos_torax_relevantes[("total_nacionales_n_egresos")].astype(int).astype(str)
    + "; "
    + round((tabla_resumen_egresos_torax_relevantes[("%_nacionales_n_egresos")] * 100), 1).astype(
        str
    )
    + "%)"
)

posiciones_torax_egresos_relevantes = tabla_resumen_egresos_torax_relevantes[
    ("ranking_nacionales_n_egresos")
].astype(int)

resumen_egresos_y_posiciones_relevantes = pd.concat(
    [resumen_egresos_torax_y_nacional_relevantes, posiciones_torax_egresos_relevantes], axis=1
)

resumen_egresos_y_posiciones_relevantes.columns = [
    "2016",
    "2017",
    "2018",
    "2019",
    "2016",
    "2017",
    "2018",
    "2019",
]

display(resumen_egresos_y_posiciones_relevantes)

,2016,2017,2018,2019,2016,2017,2018,2019
Diagnóstico,,,,,,,,
"C341 - Tumor maligno del lóbulo superior, bronquio o pulmón",180 (400; 45.0%),180 (372; 48.4%),220 (478; 46.0%),220 (493; 44.6%),1,1,1,1
Z450 - Asistencia y ajuste de marcapaso cardíaco,101 (1146; 8.8%),71 (718; 9.9%),132 (649; 20.3%),137 (685; 20.0%),2,1,1,1
J841 - Otras enfermedades pulmonares intersticiales con fibrosis,124 (1617; 7.7%),108 (1666; 6.5%),108 (1633; 6.6%),122 (1697; 7.2%),1,1,1,1
"C343 - Tumor maligno del lóbulo inferior, bronquio o pulmón",120 (241; 49.8%),118 (258; 45.7%),124 (293; 42.3%),120 (279; 43.0%),1,1,1,1
"D381 - Tumor de comportamiento incierto o desconocido de la tráquea, de los bronquios y del pulmón",91 (637; 14.3%),36 (553; 6.5%),85 (641; 13.3%),97 (654; 14.8%),1,1,1,1
J679 - Neumonitis debida a hipersensibilidad a polvo orgánico no especificado,53 (68; 77.9%),77 (106; 72.6%),70 (101; 69.3%),69 (109; 63.3%),1,1,1,1
C780 - Tumor maligno secundario del pulmón,43 (328; 13.1%),57 (373; 15.3%),63 (379; 16.6%),62 (430; 14.4%),1,1,1,1
J848 - Otras enfermedades pulmonares intersticiales especificadas,40 (98; 40.8%),25 (113; 22.1%),53 (172; 30.8%),52 (158; 32.9%),1,1,1,1
I351 - Insuficiencia (de la válvula) aórtica,19 (209; 9.1%),37 (318; 11.6%),52 (327; 15.9%),49 (313; 15.7%),1,2,1,1


In [44]:
resumen_egresos_y_posiciones_relevantes = resumen_egresos_y_posiciones_relevantes.reset_index()

In [45]:
visualize.add_dataframes_to_powerpoint(
    [resumen_egresos_y_posiciones_relevantes],
    "../reports/presentacion.pptx",
    font_size=8,
    cell_width=33,
    cell_height=0.5,
    font_family="Open Sans",
    max_cell_characters=100,
)

Para hacer un contorl de lo que se calculo, se observará una muestra de 10 hospitales - diagnósticos
distintos


In [46]:
display(rankings_nacionales.sample(10))

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_ESTABLECIMIENTO_SALUD,Capítulo,Sección,Categoría,Descripción,DIAG1,n_egresos,dias_estada_totales,n_int_q,n_muertos,ranking_nacionales_n_egresos,total_nacionales_n_egresos,%_nacionales_n_egresos,ranking_publicos_n_egresos,total_publicos_n_egresos,%_publicos_n_egresos,ranking_privados_n_egresos,total_privados_n_egresos,%_privados_n_egresos,ranking_grd_n_egresos,total_grd_n_egresos,%_grd_n_egresos,ranking_interno_n_egresos,total_interno_n_egresos,%_interno_n_egresos
i64,i64,str,str,str,str,str,str,u32,i64,i64,i64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64,u32,u32,f64
2013,104104,"""Hospital Dr. M…","""Cap.15 EMBARA…","""O60-O77 COMPL…","""O62 ANORMALIDA…","""Otras anomalía…","""O628""",1,1,0,0,7,56,0.017857,5,43,0.023256,null,null,null,null,null,null,null,null,null
2014,119207,"""Clínica Bio Bi…","""Cap.19 LESION…","""S50-S59 TRAUM…","""S52 FRACTURA D…","""Fractura de la…","""S525""",6,7,6,0,56,2574,0.002331,null,null,null,29,1069,0.005613,null,null,null,null,null,null
2002,113100,"""Hospital Barro…","""Cap.09 ENFERM…","""I30-I52 OTRAS…","""I31 OTRAS ENFE…","""Enfermedad del…","""I319""",3,8,0,1,9,126,0.02381,4,65,0.046154,null,null,null,4,61,0.04918,null,null,null
2004,121202,"""Clínica Aleman…","""Cap.19 LESION…","""S20-S29 TRAUM…","""S27 TRAUMATISM…","""Hemotórax trau…","""S271""",1,16,0,0,31,222,0.004505,null,null,null,8,26,0.038462,null,null,null,null,null,null
2013,115106,"""Hospital de Pi…","""Cap.10 ENFERM…","""J30-J39 OTRAS…","""J39 OTRAS ENFE…","""Otros abscesos…","""J391""",1,3,0,0,6,37,0.027027,3,20,0.05,null,null,null,null,null,null,null,null,null
2015,105103,"""Hospital Dr. H…","""Cap.19 LESION…","""T29-T32 QUEMA…","""T31 QUEMADURAS…","""Quemaduras que…","""T311""",1,7,1,0,22,91,0.010989,14,59,0.016949,null,null,null,null,null,null,null,null,null
2007,125100,"""Hospital Regio…","""Cap.01 CIERTA…","""A30-A49 OTRAS…","""A46 ERISIPELA""","""Erisipela""","""A46X""",1,3,0,0,100,810,0.001235,78,693,0.001443,null,null,null,44,429,0.002331,null,null,null
2017,111200,"""Hospital Clíni…","""Cap.01 CIERTA…","""B00-B09 INFEC…","""B08 OTRAS INFE…","""Exantema súbit…","""B082""",1,1,0,0,18,77,0.012987,null,null,null,8,29,0.034483,null,null,null,null,null,null
2015,122101,"""Hospital de Co…","""Cap.09 ENFERM…","""I95-I99 OTROS…","""I99 OTROS TRAS…","""Otros trastorn…","""I99X""",2,4,0,0,63,1205,0.00166,52,1068,0.001873,null,null,null,null,null,null,null,null,null


En la muestra anterior se puede observar las métricas previamente calculada, y el ranking
respectivo que tiene cada hospital para diagnósticos en cada año específico.


In [47]:
rankings_nacionales.to_pandas().to_csv(
    "../data/interim/ranking_nacional_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

rankings_hospital_analizado.to_pandas().to_csv(
    "../data/interim/ranking_torax_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## Metricas para personas sobre 14 años

En este análisis se quiere ver el ranking de egresos pero solamente tomando en cuenta a personas
mayores a 14 años. Esto, para dilucidar si el hospital del tórax es altamente relevante para tratar
patologías congénitas de adultos.


In [48]:
df_nacional_mayores_a_14 = df_nacional.filter(pl.col("EDAD_A_OS") > 14)

In [49]:
metricas_sobre_14 = build_features.obtener_metricas_egresos(
    df_nacional_mayores_a_14, AGRUPACION
).collect()

In [50]:
variables_a_rankear = ["n_egresos"]
subgrupo_del_ranking = ["ANO_EGRESO", "DIAG1"]

rankings_nacionales_sobre_14 = build_features.agregar_ranking_estratos(
    metricas_sobre_14, dict_estratos, variables_a_rankear, subgrupo_del_ranking
)

diags_congenitos_relevantes = rankings_nacionales_sobre_14.filter(
    (pl.col("DIAG1").is_in(["Q230", "Q211", "Q231"])) & (pl.col("ANO_EGRESO") == 2019)
)

In [51]:
diags_congenitos_relevantes.to_pandas().to_csv(
    "../data/interim/diags_congenitas_sobre_14_egresos.csv",
    sep=";",
    decimal=".",
    encoding="latin-1",
    index=False,
)

## 3. Análisis de procedimientos DEIS

En este análisis se quieren responder las siguientes preguntas:

1. ¿Cuál es el procedimiento más frecuente dentro de los pacientes que se les realizó un procedimiento
   quirúrgico?


In [52]:
proced = (
    df_nacional.filter(pl.col("INTERV_Q") == 1)
    .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])
    .agg(pl.count())
).collect()

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_20320\3397463805.py:3: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby(["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "GLOSA_PROCED_PPAL"])


In [53]:
display(proced)

ANO_EGRESO,ESTABLECIMIENTO_SALUD,GLOSA_PROCED_PPAL,count
i64,i64,str,u32
2001,112100,null,13867
2001,101100,null,4816
2001,112103,null,1143
2001,109101,null,4618
2001,115100,null,7926
2001,121202,null,3181
2001,120101,null,3313
2001,118200,null,1996
2001,120105,null,60


Los resultados muestran el desglose por año y recinto hospitalario junto a la cantidad de
procedimientos realizados en cada intervención quirúrgica. Sin embargo, se puede observar que
existe una cantidad importante de casos donde la glosa del procedimiento fue nula.

Ahora, se quiere calcular cuántas intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [54]:
display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_20320\989951252.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  display(proced.groupby("GLOSA_PROCED_PPAL").agg(pl.col("count").sum()).sort("count"))


GLOSA_PROCED_PPAL,count
str,u32
"""Resonancia Mag…",1
"""Spect - Tomogr…",1
"""Respuesta resp…",1
"""Radiografía re…",1
"""Extraccion per…",1
"""E.E.G. Post-pr…",1
"""Mamografía pro…",1
"""Sondeo gastric…",1
"""Criocoagulacio…",1


Por lo tanto, 12382642 de todas las intervenciones quirúrgicas carecen de una glosa de procedimiento.


In [55]:
proced.write_excel("../data/interim/conteo_procedimientos.xlsx")